In [2]:

import boto3
import time
from datetime import timedelta
import sys
import json
import os


AWS_ACCESS_KEY_ID ="AWS_ACCESS_KEY_ID"
AWS_SECRET_ACCESS_KEY = "AWS_SECRET_ACCESS_KEY"
AWS_REGION = "ap-northeast-1"

# sys.stdout = open('rekognition_output.txt','w')
start_time = time.monotonic()

BUCKET = "product"
KEY = "2024-03-13 100407.png"

def detect_labels(bucket=BUCKET, key=KEY, max_labels = 10, min_confidence = 90, region=AWS_REGION):
	rekognition = boto3.client("rekognition", region, aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY) 
	response = rekognition.detect_labels(
		Image = {
			"S3Object": {
				"Bucket": bucket,
				"Name": key,
			}
		},
		MaxLabels = max_labels,
		MinConfidence = min_confidence,
	)
	return response['Labels']

print('Detected Labels for:' + KEY)
for label in detect_labels(bucket=BUCKET, key=KEY):
	print("{Name} - {Confidence}%".format(**label))
print('\n')


Detected Labels for:螢幕擷取畫面 2024-03-13 100407.png
Clothing - 99.99970245361328%
Hat - 99.99970245361328%
Sun Hat - 99.99957275390625%




In [39]:
#多張一起 保存原始檔以及處理好資料.json

import boto3
import time
from datetime import timedelta
import json

AAWS_ACCESS_KEY_ID ="AWS_ACCESS_KEY_ID"
AWS_SECRET_ACCESS_KEY = "AWS_SECRET_ACCESS_KEY"
AWS_REGION = "ap-northeast-1"

BUCKET = "product"

def detect_labels_and_colors(bucket, key, max_labels=10, min_confidence=90, region=AWS_REGION):
    rekognition = boto3.client("rekognition", region,
                               aws_access_key_id=AWS_ACCESS_KEY_ID,
                               aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
    
    # 檢測標籤和顏色
    response = rekognition.detect_labels(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        MaxLabels=max_labels,
        MinConfidence=min_confidence,
        Features=['GENERAL_LABELS', 'IMAGE_PROPERTIES']
    )

    # print(response)
        
    # 提取標籤和顏色信息
    labels = response.get('Labels', [])
    colors = response.get('ImageProperties', [])
    
    return response, labels, colors

# def list_first_three_images(bucket):
#     s3 = boto3.client("s3",
#                       aws_access_key_id=AWS_ACCESS_KEY_ID,
#                       aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
#     paginator = s3.get_paginator('list_objects_v2')
#     pages = paginator.paginate(Bucket=bucket)
#     count = 0
#     for page in pages:
#         if 'Contents' in page:
#             for obj in page['Contents']:
#                 if count < 3:
#                     yield obj['Key']
#                     count += 1
#                 else:
#                     break

def list_all_images(bucket):
    s3 = boto3.client("s3",
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket)
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                yield obj['Key']

def main():
    original_response_list = []
    output_response_list = []
    with open('rekognition_output.txt', 'w') as output_file:
        for image_key in list_all_images(BUCKET):
            labels_list = []
            colors_list = []
            single_output_json = {}
            print('Detecting labels and colors for:', image_key)
            response, labels, colors = detect_labels_and_colors(bucket=BUCKET, key=image_key)
            original_response_list.append({"image_key": image_key, "response": response})  # 將圖片相關資訊添加到原始回應的 JSON 中
            output_file.write('Detected Labels and Colors for: {}\n'.format(image_key))
            output_file.write('Labels:\n')
            for label in labels:
                label_output = {
                    "Name": label['Name'],
                    "Confidence": label['Confidence'],
                    "Parents": [parent['Name'] for parent in label.get('Parents', [])],
                    "Categories": [category['Name'] for category in label.get('Categories', [])]
                }
                labels_list.append(label_output)
                output_file.write(" - Label: {} (Confidence: {}%)\n".format(label['Name'], label['Confidence']))
                output_file.write("   Parents:\n")
                for parent in label.get('Parents', []):
                    output_file.write("     - {}\n".format(parent['Name']))
                output_file.write("   Categories:\n")
                for category in label.get('Categories', []):
                    output_file.write("     - {}\n".format(category['Name']))

            # 寫入顏色信息
            if 'Foreground' in colors:
                colors_info = colors['Foreground']
            else:
                colors_info = colors
            
            if 'DominantColors' in colors_info:
                colors_data = colors_info['DominantColors']
                output_file.write('Colors:\n')
                for color_info in colors_data:
                    color_output = {
                        "RGB": [color_info['Red'], color_info['Green'], color_info['Blue']],
                        "HexCode": color_info['HexCode'],
                        "CSSColor": color_info.get('CSSColor', ''),
                        "SimplifiedColor": color_info.get('SimplifiedColor', ''),
                        "Pixel Percentage": color_info['PixelPercent']
                    }
                    colors_list.append(color_output)
                    output_file.write(" - RGB: ({}, {}, {})\n".format(color_info['Red'], color_info['Green'], color_info['Blue']))
                    output_file.write("   HexCode: {}\n".format(color_info['HexCode']))
                    output_file.write("   CSSColor: {}\n".format(color_info.get('CSSColor', '')))
                    output_file.write("   SimplifiedColor: {}\n".format(color_info.get('SimplifiedColor', '')))
                    output_file.write("   Pixel Percentage: {}%\n".format(color_info['PixelPercent']))
            
            single_output_json = {"Detecting image": image_key, "Properties": [{"Labels": labels_list, "Colors": colors_list}]}
            output_response_list.append(single_output_json)
            output_file.write('\n')
            output_file.flush()

        with open('original_response.json', 'w') as original_file:
            json.dump(original_response_list, original_file, indent=2)
        with open('output_response.json', 'w') as original_file:
            json.dump(output_response_list, original_file, indent=2)
            
# def main():
#     with open('rekognition_output.txt', 'w') as output_file:
#         for image_key in list_all_images(BUCKET):
#             print('Detecting labels for:', image_key)
#             labels = detect_labels(bucket=BUCKET, key=image_key)
#             output_file.write('Detected Labels for: {}\n'.format(image_key))
#             for label in labels:
#                 output_file.write("{Name} - {Confidence}%\n".format(**label))
#             output_file.write('\n')
#             output_file.flush()  # Ensure the content is written immediately

if __name__ == "__main__":
    start_time = time.monotonic()
    main()
    end_time = time.monotonic()
    print("Execution time:", timedelta(seconds=end_time - start_time))


Detecting labels and colors for: aws_pic_test/6548504.jpg
Detecting labels and colors for: aws_pic_test/6548505.jpg
Detecting labels and colors for: aws_pic_test/6548506.jpg
Detecting labels and colors for: aws_pic_test/6548507.jpg
Detecting labels and colors for: aws_pic_test/6548508.jpg
Detecting labels and colors for: aws_pic_test/6548509.jpg
Detecting labels and colors for: aws_pic_test/6548510.jpg
Detecting labels and colors for: aws_pic_test/6548511.jpg
Detecting labels and colors for: aws_pic_test/6548513.jpg
Detecting labels and colors for: aws_pic_test/6548514.jpg
Detecting labels and colors for: aws_pic_test/6548515.jpg
Detecting labels and colors for: aws_pic_test/6548516.jpg
Detecting labels and colors for: aws_pic_test/6548517.jpg
Detecting labels and colors for: aws_pic_test/6548518.jpg
Detecting labels and colors for: aws_pic_test/6548519.jpg
Detecting labels and colors for: aws_pic_test/6548521.jpg
Detecting labels and colors for: aws_pic_test/6548522.jpg
Detecting labe